In [145]:
import numpy as np
import pandas as pd

In [146]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [147]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [148]:
train_df_len = len(train_df)
test_df_len = len(test_df)

In [149]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Data Cleaning
1. Wich features are categorical and wich are numerical?
2. Available Data Types?
3. None, blanck or empty values?

In [150]:
import matplotlib.pyplot as plt
import seaborn as sns

In [151]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [152]:
# garantiza que los cambios en se apliquen de manera uniforme a ambos conjuntos.
# Los DF se pueden separar facilmente para entrenar y evaluar el modelo.

def combine_df(train_df, test_df):
    combined_df = pd.concat([train_df, test_df], ignore_index=True)
    return combined_df

def separate_df(combined_df, train_df_len, test_df_len):
    train_df = combined_df.iloc[:train_df_len]
    test_df = combined_df.iloc[test_df_len:]
    return train_df, test_df

full_df = combine_df(train_df, test_df)
len(full_df)

12970

In [153]:
# imputacion de valores faltantes

zero_columns = ['RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck']
false_columns = ['VIP']
condition = (full_df['CryoSleep'] == True)

full_df.loc[~condition, zero_columns] = full_df.loc[~condition, zero_columns].fillna(0)
full_df.loc[~condition, false_columns] = full_df.loc[~condition, zero_columns].fillna(False)

In [154]:
zero_columns = ['RoomService', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck']
cryosleep_columns = ['CryoSleep']

condition = (full_df[zero_columns].eq(0).all(axis=1))

full_df.loc[~condition, cryosleep_columns] = full_df.loc[~condition, cryosleep_columns].fillna(True)

/tmp/ipython-input-616523900.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_df.loc[~condition, cryosleep_columns] = full_df.loc[~condition, cryosleep_columns].fillna(True)


In [155]:
full_df['CryoSleep'].fillna(False, inplace=True)

/tmp/ipython-input-2329576505.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df['CryoSleep'].fillna(False, inplace=True)
/tmp/ipython-input-2329576505.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_df['CryoSleep'].fillna(False, inplace=True)


In [156]:
full_df.isna().sum()

,0
PassengerId,0
HomePlanet,288
CryoSleep,0
Cabin,299
Destination,274
Age,270
VIP,8493
RoomService,93
FoodCourt,109
ShoppingMall,131


In [157]:
full_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,NaN,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,NaN,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,NaN,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,NaN,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,NaN,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [158]:
cabin_split = full_df['Cabin'].str.split('/', expand = True)
cabin_split.columns = ['Deck', 'Room_Number', 'Side']
new_df = pd.concat([full_df, cabin_split], axis=1)
full_df = new_df
full_df.drop(columns=['Cabin'], inplace=True)
full_df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Room_Number,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,NaN,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,NaN,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,NaN,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,NaN,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,NaN,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


In [159]:
group_means = full_df.groupby(['HomePlanet', 'CryoSleep'])['Age'].mean()
group_means

HomePlanet  CryoSleep
Earth       False        27.056735
            True         23.575071
Europa      False        35.488731
            True         32.929037
Mars        False        29.800888
            True         28.934426
Name: Age, dtype: float64

In [160]:
# Rellenar valores NaN en Age

group_means = full_df.groupby(['HomePlanet', 'CryoSleep'], group_keys = True)['Age'].mean().reset_index(name= 'Age_mean')
full_df = full_df.merge(group_means, on=['HomePlanet', 'CryoSleep'], how='left')
full_df['Age'] = full_df['Age'].fillna(full_df['Age_mean'])
full_df.drop(columns=['Age_mean'], inplace=True)

In [161]:
full_df.isna().sum()

,0
PassengerId,0
HomePlanet,288
CryoSleep,0
Destination,274
Age,5
VIP,8493
RoomService,93
FoodCourt,109
ShoppingMall,131
Spa,107


In [162]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [163]:
numerical_features = ['Room_Number', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, numerical_features),
    ('categorical', categorical_pipeline, categorical_features)
])

In [164]:
train_df, test_df = separate_df(full_df, train_df_len, test_df_len)
y = train_df['Transported']
train_df.drop(columns=['Transported'], inplace=True)
test_df.drop(columns=['Transported'], inplace=True)

y= y.astype(bool)
y.head()

/tmp/ipython-input-2840458369.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns=['Transported'], inplace=True)
/tmp/ipython-input-2840458369.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns=['Transported'], inplace=True)


,Transported
0,False
1,True
2,False
3,False
4,True


In [165]:
transformed_data_train = preprocessor.fit_transform(train_df)
transformed_data_test = preprocessor.transform(test_df)

feature_names = preprocessor.named_transformers_['categorical']\
    .named_steps['encoder'].get_feature_names_out(input_features = categorical_features)

all_feature_names = numerical_features + list(feature_names)

In [166]:
transformed_train_df = pd.DataFrame(transformed_data_train, columns=all_feature_names)
transformed_test_df = pd.DataFrame(transformed_data_test, columns=all_feature_names)

In [167]:
transformed_train_df.head()

,Room_Number,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,-1.186627,0.709082,-0.335878,-0.283399,-0.286874,-0.272740,-0.264958,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-1.186627,-0.336318,-0.170774,-0.277757,-0.245047,0.215179,-0.226150,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,-1.186627,2.033254,-0.270745,1.958344,-0.286874,5.695155,-0.221740,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.186627,0.290922,-0.335878,0.520895,0.333834,2.685879,-0.094731,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-1.184651,-0.893864,0.123080,-0.239517,-0.034241,0.229399,-0.263194,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [168]:
SEED = 42

In [169]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [170]:
gbm_model_1 = GradientBoostingClassifier(n_estimators=70, learning_rate=0.1, max_features='sqrt', max_depth=5, random_state=SEED,
                                         min_samples_split=2, min_samples_leaf=3, loss='exponential', subsample = 0.5)
gbm_model_2 = GradientBoostingClassifier(n_estimators=70, learning_rate=0.1, max_features='log2', max_depth=5, random_state=SEED,
                                         min_samples_split=2, min_samples_leaf=3, loss='log_loss', subsample = 0.5)
gbm_model_3 = GradientBoostingClassifier(n_estimators=70, learning_rate=0.1, max_features='log2', max_depth=5, random_state=SEED,
                                         min_samples_split=2, min_samples_leaf=3, loss='exponential', subsample = 0.5)


In [171]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transformed_train_df, y, test_size=0.2, random_state=SEED)

In [172]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [173]:
gbm_model_1.fit(X_train, y_train)
gbm_model_2.fit(X_train, y_train)
gbm_model_3.fit(X_train, y_train)

gbm_1_predictions = gbm_model_1.predict(X_test)
gbm_2_predictions = gbm_model_2.predict(X_test)
gbm_3_predictions = gbm_model_3.predict(X_test)

stacked_features = np.column_stack((gbm_1_predictions, gbm_2_predictions, gbm_3_predictions))

meta_model = LogisticRegression()
meta_model.fit(stacked_features, y_test)

gbm_1_base_preds = gbm_model_1.predict(X_test)
gbm_2_base_preds = gbm_model_2.predict(X_test)
gbm_3_base_preds = gbm_model_3.predict(X_test)

stacked_base_preds = np.column_stack((gbm_1_base_preds, gbm_2_base_preds, gbm_3_base_preds))

ensemble_predictions = meta_model.predict(stacked_base_preds)

ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)

print("Ensemble Accuracy:", ensemble_accuracy)

Ensemble Accuracy: 0.7952846463484762


In [174]:
gbm_1_base_preds = gbm_model_1.predict(transformed_test_df)
gbm_2_base_preds = gbm_model_2.predict(transformed_test_df)
gbm_3_base_preds = gbm_model_3.predict(transformed_test_df)

stacked_base_preds = np.column_stack((gbm_1_base_preds, gbm_2_base_preds, gbm_3_base_preds))

In [175]:
ensemble_predictions = meta_model.predict(stacked_base_preds)

In [176]:
ensemble_predictions

array([False, False, False, ...,  True,  True,  True])

In [177]:
output = pd.DataFrame({'PassengerId': test_df['PassengerId'],
                       'Transported': ensemble_predictions})

In [178]:
output.to_csv('submission.csv', index=False)
print('Submission saved')

Submission saved
